In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-31"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18464,2024-03-31,Japão B.League,02:05,Nagasaki,Shimane,2.72,1.42,163.5,1.85,1.85,0.0,0.00,0.0,AmwiTztR,0.367647,0.704225,0.540541,0.540541,0.071872,194.840,53.157255,0.272825,1.2,1.643168,1.369306,207.20,2.618,0.669940,0.255898,-29.0,139.780,61.884719,0.442729,1.8,1.643168,0.912871,93.80,1.636,0.398095,0.243335,31.0,74,67,2.80,1.40,227.100,183.588,0.444077,0.000000,NaN,1.78,0.356,4.831461,0.382682,0.2,-0.182682,0.49,0.098,4.285714,0.546009,0.6,0.053991
18465,2024-03-31,Japão B.League,02:05,Hiroshima D.,Gunma,1.66,2.13,157.5,1.80,1.90,0.0,0.00,0.0,2NZrVdBE,0.602410,0.469484,0.555556,0.526316,0.071893,127.914,28.387664,0.221928,2.4,1.341641,0.559017,138.00,1.606,0.282984,0.176204,17.0,154.160,78.204275,0.507293,1.2,1.643168,1.369306,212.42,1.846,0.851780,0.461420,8.0,92,86,1.50,2.47,157.858,176.398,0.175377,0.038222,NaN,0.25,0.050,13.200000,0.603310,0.5,-0.103310,2.33,0.466,2.424893,0.617920,0.9,0.282080
18466,2024-03-31,Japão B.League,06:05,Ryukyu,Ibaraki Robots,1.08,6.90,161.5,1.80,1.90,0.0,0.00,0.0,GUFJOflr,0.925926,0.144928,0.555556,0.526316,0.070853,95.072,4.414648,0.046435,2.4,1.341641,0.559017,101.92,1.136,0.091815,0.080823,38.0,396.576,294.963667,0.743776,0.6,1.341641,2.236068,864.00,4.430,3.059828,0.690706,-32.0,98,96,1.04,9.00,99.936,328.142,1.031419,0.038222,NaN,-0.63,-0.126,-0.634921,0.858112,0.7,-0.158112,1.13,0.226,26.106195,0.291882,0.5,0.208118
18467,2024-03-31,Japão B.League,02:05,Osaka,Hokkaido,1.13,5.45,163.5,1.83,1.87,0.0,0.00,0.0,OWzuWxd8,0.884956,0.183486,0.546448,0.534759,0.068442,223.344,198.574833,0.889099,2.4,1.341641,0.559017,112.32,2.734,2.571581,0.940593,38.0,262.058,115.019532,0.438909,0.6,1.341641,2.236068,344.84,3.538,1.892926,0.535027,-54.0,96,74,1.17,4.66,181.116,432.248,0.928481,0.015289,NaN,0.40,0.080,1.625000,0.486443,0.6,0.113557,-5.00,-1.000,-4.450000,0.255557,0.2,-0.055557
18468,2024-03-31,Itália Série A2,13:00,Rimini,Monferrato,1.23,3.84,157.5,1.85,1.85,0.0,0.00,0.0,MkqbYtBg,0.813008,0.260417,0.540541,0.540541,0.073425,185.596,105.420408,0.568010,2.4,1.341641,0.559017,149.76,2.294,1.247109,0.543639,29.0,211.234,33.702774,0.159552,0.6,1.341641,2.236068,187.50,2.642,0.322289,0.121987,-14.0,64,75,2.34,2.50,121.838,218.584,0.728027,0.000000,NaN,2.54,0.508,0.452756,0.638690,0.7,0.061310,-3.29,-0.658,-4.316109,0.383838,0.1,-0.283838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18606,2024-03-31,Coreia Do Sul Kbl,06:00,Goyang Sono,Suwon KT,1.60,2.32,171.5,1.80,1.92,-4.5,2.01,2.2,rJb0SEeo,0.625000,0.431034,0.555556,0.520833,0.056034,248.404,59.623739,0.240027,1.8,1.643168,0.912871,217.56,3.128,0.956096,0.305657,-4.0,135.880,49.654118,0.365426,1.8,1.643168,0.912871,103.68,1.468,0.411424,0.280262,8.0,84,81,2.59,1.28,211.368,204.866,0.259754,0.045620,0.063824,0.25,0.050,12.000000,0.444658,0.4,-0.044658,-2.91,-0.582,-2.268041,0.587043,0.4,-0.187043
18607,2024-03-31,Coreia Do Sul Kbl,06:00,KCC Egis,Seoul Knights,1.54,2.45,168.5,1.84,1.94,-5.5,2.04,2.9,YqDUXzYN,0.649351,0.408163,0.543478,0.515464,0.057514,148.036,41.458656,0.280058,1.8,1.643168,0.912871,157.30,1.554,0.451807,0.290738,5.0,121.8

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
18465,02:05,Japão B.League,Hiroshima D.,Gunma,1.66,Back Home
18474,03:05,Japão B.League,Chiba,Brave Thunders,2.29,Back Home
18489,06:00,Geórgia Superliga,Gau,Batumi,1.61,Back Home
18497,02:00,Japão B2.League,Kobe,Shizuoka,1.62,Back Home
18511,19:00,Chile Lnb,Deportes Castro,Valdivia,1.93,Back Home
18533,12:00,Romênia Divisão A,Rapid Bucuresti,CSU Sibiu,1.54,Back Home
18540,13:30,Alemanha Bbl,Rostock,Oldenburg,2.17,Back Home
18555,20:00,Eua Nba,New York Knicks,Oklahoma City Thunder,1.75,Back Home
18569,20:00,Eua Nba,Houston Rockets,Dallas Mavericks,2.05,Back Home
